In [46]:
import pandas as pd
import xgboost
import shap
import gately
import numpy as np
import random
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [47]:
from sktime.classification.kernel_based import RocketClassifier
from sktime.datasets import load_gunpoint
from sktime.datasets import load_italy_power_demand

X, y = load_italy_power_demand()


numpy_X = []
for i in range(0,X.shape[0]):
    numpy_X.append(X.loc[i].values[0])
X = np.asarray(numpy_X)

y = np.asarray(y)

#X = (X - np.min(X))/(np.max(X)-np.min(X))
#y =  (y - np.min(y))/(np.max(y)-np.min(y))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [48]:
X_train.shape

(734, 24)

In [49]:
reg = RocketClassifier(num_kernels=100)
reg.fit(np.asarray(X_train), np.asarray(y_train))
clf = reg
clf.predict_proba(X_test[3].reshape(1,24))[0][0]

1.0

In [50]:
clf.score(X_test,y_test)


0.9640883977900553

In [51]:
explainer = shap.KernelExplainer(clf.predict_proba, X_train[0].reshape(1,-1))
shap_values = explainer.shap_values(X_test[1].reshape(1,-1), nsamples=1000)

  0%|          | 0/1 [00:00<?, ?it/s]

The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight

In [52]:
shap_values[0][0]

array([ 0.03590847, -0.02056493,  0.05548957,  0.02319163,  0.        ,
        0.01961935,  0.04153908,  0.02621211,  0.05586956,  0.00891495,
       -0.0245736 , -0.01778603,  0.01622995, -0.01529149,  0.09811942,
        0.        , -0.05264054, -0.02081208,  0.27196448,  0.25501801,
        0.10235057,  0.02337496,  0.07849563,  0.03937093])

In [71]:
classA = []
classB = []
for i in X_test:
    if clf.predict(i.reshape(1,-1)) == '1':
        classB.append(i)
    else:
        classA.append(i)

In [72]:


matches = 0
gate_distance = 0
ks_distance = 0
total = 0
for ind in range(0,50):
    explainer = shap.KernelExplainer(clf.predict_proba, classA[ind].reshape(1,-1))
    shap_values = explainer.shap_values(classB[ind].reshape(1,-1), nsamples=100)
    att_gately = gately.Gately_classification(classB[ind],classA[ind],clf.predict_proba)
    
    
    if att_gately == -1:
        print("skip")
    else:
   
        ks = {}
        for i in range(0,X.shape[1]):
            ks[i] = np.abs(shap_values[0][0][i])

        gat = {}
        selected_inds = []
        for i in range(0,X.shape[1]):
            gat[i] = np.abs(att_gately[i])
            if gat[i] > 0:
                selected_inds.append(i)

        tick = 0


        for i,j in zip(ks.values(),att_gately.values()):
            if round(i,3) != round(j,3):
                tick = 1
        if tick == 0:
            matches += 1
        
        print(selected_inds)



        ks_sorted = sorted(ks, key=ks.get, reverse=True)
        gate_sorted = sorted(gat, key=gat.get, reverse=True)

        x_sample = classB[ind].copy()
        x_sample[ks_sorted[0:len(selected_inds)]] = classA[ind][ks_sorted[0:len(selected_inds)]]
        ks_prediction = clf.predict_proba(x_sample.reshape(1,-1))[0]

        x_sample = classB[ind].copy()
        x_sample[selected_inds] = classA[ind][selected_inds]
        gate_prediction = clf.predict_proba(x_sample.reshape(1,-1))[0]

        print("ORIGINAL PREDICTION")
        original_prediction = clf.predict_proba(classA[ind].reshape(1,-1))[0]
        print(clf.predict_proba(classA[ind].reshape(1,-1))[0])
        print("GATE")
        print(gate_prediction[0] - original_prediction[0])
        print("KS")
        print(ks_prediction[0] - original_prediction[0])
        
        if gate_prediction[0] - original_prediction[0] == 0:
            gate_distance += 1
        if ks_prediction[0] - original_prediction[0] == 0:
            ks_distance += 1
            
        #gate_distance.append(np.abs(gate_prediction - clf.predict_proba(classA[ind].reshape(1,-1))[0]))
        #ks_distance.append(np.abs(ks_prediction - clf.predict_proba(classA[ind].reshape(1,-1))[0]))
        total += 1


  0%|          | 0/1 [00:00<?, ?it/s]

The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight

cant compute
skip


  0%|          | 0/1 [00:00<?, ?it/s]

The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight

...

The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight

[19, 20]
ORIGINAL PREDICTION
[0. 1.]
GATE
0.0
KS
0.0


  0%|          | 0/1 [00:00<?, ?it/s]

The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight

cant compute
skip


In [69]:
gate_distance/total

0.6285714285714286

In [70]:
ks_distance/total

0.5142857142857142

In [58]:
gate_distance

0

In [ ]:
print(np.mean(gate_distance))
print(np.std(gate_distance))

In [ ]:
print(np.mean(ks_distance))
print(np.std(ks_distance))

In [ ]:
ks_distance

In [ ]:
ks

In [ ]:
gat

In [ ]:
x_sample = classB[ind].copy()
x_sample[gate_sorted[0]] = classA[ind][gate_sorted[0]]
gate_prediction = clf.predict_proba(x_sample.reshape(1,-1))[0]

In [ ]:
np.abs(gate_prediction - clf.predict_proba(classA[ind].reshape(1,-1))[0])

In [ ]:
plt.plot(classA[ind])
for i in range(0,24):
    plt.axvline(i,color='blue',alpha=ks[i])
plt.xlabel('Observation Index')
plt.ylabel('Power Demand')
plt.savefig('Italy_ref_ks.pdf')

In [ ]:
plt.plot(classA[ind])
for i in range(0,24):
    plt.axvline(i,color='red',alpha=gat[i])
plt.xlabel('Observation Index')
plt.ylabel('Power Demand')
plt.savefig('Italy_ref_gat.pdf')

In [ ]:
plt.plot(classB[ind])
for i in range(0,24):
    plt.axvline(i,color='blue',alpha=ks[i])
plt.xlabel('Observation Index')
plt.ylabel('Power Demand')
plt.savefig('Italy_sample_ks.pdf')

In [ ]:
plt.plot(classB[ind])
for i in range(0,24):
    plt.axvline(i,color='red',alpha=gat[i])
plt.xlabel('Observation Index')
plt.ylabel('Power Demand')
plt.savefig('Italy_sample_gat.pdf')